In [3]:
import pandas as pd
from glob import glob
import os
import os.path as osp 

In [4]:
path_to_csv = r'D:\Project- Electro\VAME\materials\working_dir\Unsupervised Learning Tutorial with VAME-May28-2024\videos\pose_estimation_raw'
save_path = r'D:\Project- Electro\VAME\materials\working_dir\Unsupervised Learning Tutorial with VAME-May28-2024\videos\pose_estimation'
csv_files = glob(osp.join(path_to_csv, '*csv'))

In [49]:
def convert_to_multilevel_header(df):
    # Extract existing column names
    old_columns = df.columns
    
    # Initialize lists for new header levels
    # scorer = []
    individuals = []
    bodyparts = []
    coords = []
    
    # Loop through old columns to populate new header levels
    for col in old_columns:
        # Split the old column name by '_'
        parts = col.split('_')
        
        # Handle the different possible formats
        if len(parts) == 3:
            bodypart = parts[0]
            individual = parts[1]
            coord = parts[2]
        elif len(parts) > 3:
            bodypart = '_'.join(parts[:-2])
            individual = parts[-2]
            coord = parts[-1]
        
        # Append fixed scorer value and extracted parts to corresponding lists
        # scorer.append("DLC_resnet50_FreeRatJul12shuffle1_100000")
        individuals.append(individual)
        bodyparts.append(bodypart)
        if coord == 'p':
            coord = 'likelihood'
        coords.append(coord)
    
    # Create multi-level columns using the lists
    # multi_level_columns = pd.MultiIndex.from_arrays(
    #     [scorer, individuals, bodyparts, coords],
    #     names=['scorer', 'individuals', 'bodyparts', 'coords']
    # )
    multi_level_columns = pd.MultiIndex.from_arrays(
        [individuals, bodyparts, coords],
        names=['individuals', 'bodyparts', 'coords']
    )
    
    # Assign new multi-level columns to the dataframe
    df.columns = multi_level_columns
    
    return df

In [50]:

list_of_df = []
for csv in csv_files:
    df_temp = pd.read_csv(csv)
    df_temp.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
    # get list of pose columns
    # split pose columns between subject and stim
    all_columns = df_temp.columns
    subject_pose_columns = [a for a in all_columns if a.endswith('1_x') or a.endswith('1_y') or a.endswith('1_p')]
    stim_pose_columns = [a for a in all_columns if a.endswith('2_x') or a.endswith('2_y') or a.endswith('2_p')]
    
    # extract columns
    df_temp_subject = df_temp[subject_pose_columns]
    df_temp_stim = df_temp[stim_pose_columns]
    
    df_temp_subject = convert_to_multilevel_header(df_temp_subject)
    df_temp_stim = convert_to_multilevel_header(df_temp_stim)
    
    
    fname = osp.basename(csv).split('.csv')[0]
    
    
    df_temp_subject.to_csv(osp.join(save_path, f'{fname}_subject.csv'), index=True, header=True)
    # df_temp_stim.to_csv(osp.join(save_path, f'{fname}_stim.csv'), index=True, header=True)
    
     
    

In [53]:
path_to_csv = osp.join(save_path, f'{fname}_subject.csv')

data = pd.read_csv(path_to_csv, skiprows = 2)
data_mat = pd.DataFrame.to_numpy(data)
data_mat = data_mat[:,1:]

In [54]:
data_mat

array([[332.02421679, 372.56170241,   0.99963746, ..., 229.95574964,
        306.04500147,   0.99451139],
       [337.70182881, 373.05082486,   0.97771844, ..., 237.92890977,
        303.27374007,   0.9935206 ],
       [340.8566019 , 373.17491224,   0.96959859, ..., 240.94740074,
        302.45293562,   0.99340663],
       ...,
       [287.68687601, 688.61908133,   0.        , ..., 205.47126456,
        416.15721809,   0.        ],
       [287.68687601, 688.61908133,   0.        , ..., 205.47126456,
        416.15721809,   0.        ],
       [287.68687601, 688.61908133,   0.        , ..., 205.47126456,
        416.15721809,   0.        ]])